#### action
文本抄袭自动检测分析:
如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具
1）定义可能抄袭的文章来源
2）与原文对比定位抄袭的地方
原始数据：sqlResult.csv，共计89611篇
从数据库导出的文章，字段包括：id, author, source, content, feature, title, url
常用中文停用词：chinese_stopwords.txt

In [4]:
import pandas as pd
import numpy as np
import jieba


In [5]:
#数据加载与预览

artical = pd.read_csv('E:/bi_course/biCourse/L3/tue_course/sqlResult.csv', encoding = 'gb18030')
print(artical.shape)

#文章内容为空的数量
print("文章内容为空的数量:", len(artical[artical.content.isna()]) )

#删除文章内容为空
artical.dropna(subset = ['content'], inplace = True)
print("删除空内容的文章后数量:", artical.shape[0])

(89611, 7)
文章内容为空的数量: 2557
删除空内容的文章后数量: 87054


In [6]:
#对文章进行分词处理

f = open("E:/bi_course/biCourse/L3/tue_course/chinese_stopwords.txt","r", encoding = 'utf-8')
stopwords = [ word[:-1] for  word in f.readlines()] #[:-1]表示删除\n字符

def split_text(text):
    text = text.replace(' ','').replace('\r','').replace('\n','') #\r 回车, \n换行
    words = jieba.cut(text) 
    result  = ' '.join([w for  w in words if w not in stopwords] )
    return result

In [7]:
corpus = list( artical.content.apply( lambda x : split_text(x)) )

Building prefix dict from the default dictionary ...
Loading model from cache D:\UserData\Zoni\Temp\jieba.cache
Loading model cost 1.003 seconds.
Prefix dict has been built succesfully.


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#计算语料库corpus的TD-IDF

cv = CountVectorizer(encoding = 'gb18030', min_df= 0.015)
tfiftransformer = TfidfTransformer()
contervector = cv.fit_transform(corpus)
# print(contervector)
tfdif = tfiftransformer.fit_transform(contervector)


In [36]:
#标记是否为新华社的新闻
label = list(artical.source.apply(lambda x: 1 if x == '新华社'  else 0))

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

X_train, X_test, y_train, y_test = train_test_split(tfdif.toarray(), label, train_size = 0.7)
clf = BernoulliNB()

clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)


print('准确率:', round(accuracy_score(y_test, y_predict),4))
print('精确率:', round(precision_score(y_test, y_predict),4))
print('召回率:', round(recall_score(y_test, y_predict),4))

准确率: 0.83
精确率: 0.9649
召回率: 0.8423


In [3]:
#使用模型进行风格预测

prediction = clf.predict(tfdif.toarray())
actual = np.array(label)

#比较预测风格和是否为新华社文章:
comparison = pd.DataFrame({'articalId':artical.id,'prediction':prediction, 'actual':actual})
comparison['copy'] = 0
comparison.loc[(comparison['prediction'] == 1 ) & (comparison['actual'] == 0), 'copy'] = 1 #预测文章是新华社 实际不是新华社, 则有抄袭嫌疑
copy_article = comparison[comparison['copy'] == 1]
xinhuashe_article = comparison[comparison['actual'] == 1]

print('疑是抄袭文章数量:', len(copy_article))

NameError: name 'clf' is not defined

In [57]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfdif.toarray())

#使用聚类对全部文章进行聚类: 分为30类

km = KMeans( n_clusters= 30)
k_labels = km.fit_predict(scaled_array)
print(k_labels.shape)

(87054,)


In [2]:
copy_article.index

NameError: name 'copy_article' is not defined

In [71]:
#创建id_class

id_class = {index: class_ for index, class_ in enumerate (k_labels)} #文章经过kmeans的分类

from collections import defaultdict
class_id = defaultdict(set)

for index, class_ in id_class.items():
    if index in xinhuashe_article.index.to_list():
        class_id[class_].add(index)

In [1]:
# class_id

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#查找相似文章

def search_similar_article ( cpindex, top = 10):
    #只在新华社发布的文章中找
    dist_dict = {i: cosine_similarity(tfdif[cpindex], tfidf[i]) for i in class_id [id_class[cpindex]]}
    return sorted (dist_dict.items(), key = lambda x:x[1][0], reverse= True)[:top]



cpindex = 

similar_list = search_similar_article(cpindex)
print(similar_list)



In [ ]:
cp

In [ ]:
print('怀疑抄袭:\n', artical.iloc[cpindex].content)

#找一篇相似的原文
similar2 = similar_list[0][0]
print('相似的原文:\n', news.iloc[similar2].content)